# Content with notebooks

You can also create content with Jupyter Notebooks. This means that you can include
code blocks and their outputs in your book.

## Markdown + notebooks

As it is markdown, you can embed images, HTML, etc into your posts!

![](https://myst-parser.readthedocs.io/en/latest/_static/logo-wide.svg)

You can also $add_{math}$ and

$$
math^{blocks}
$$

or

$$
\begin{aligned}
\mbox{mean} la_{tex} \\ \\
math blocks
\end{aligned}
$$

But make sure you \$Escape \$your \$dollar signs \$you want to keep!

## MyST markdown

MyST markdown works in Jupyter Notebooks as well. For more information about MyST markdown, check
out [the MyST guide in Jupyter Book](https://jupyterbook.org/content/myst.html),
or see [the MyST markdown documentation](https://myst-parser.readthedocs.io/en/latest/).

## Code blocks and outputs

Jupyter Book will also embed your code blocks and output in your book.
For example, here's some sample Matplotlib code:

In [1]:
import numpy as np

In [2]:
import data
from model import MyRNNModel
import torch
from torch import nn
import time
import math

In [3]:
seed = 14584234
train_batch_size = 250
sequence_length = 5
p_dropout = .0
size_embedding = 50
n_hidden_layers = 50
n_rnn_layers = 1

In [4]:
# Set the random seed manually for reproducibility.
torch.manual_seed(seed)
if torch.cuda.is_available():
    device = torch.device("cuda")
elif torch.mps.is_available():
    device = torch.device("mps")
else:
    device = torch.device("cpu")
print('Using %s device' % device)

Using cuda device


In [5]:
corpus = data.Corpus('ptb_data')

def batchify(data, bsz):
    # Work out how cleanly we can divide the dataset into bsz parts.
    nbatch = data.size(0) // bsz
    # Trim off any extra elements that wouldn't cleanly fit (remainders).
    data = data.narrow(0, 0, nbatch * bsz)
    # Evenly divide the data across the bsz batches.
    data = data.view(bsz, -1).t().contiguous()
    return data.to(device)

eval_batch_size = 10
train_data = batchify(corpus.train, train_batch_size)
val_data = batchify(corpus.valid, eval_batch_size)
test_data = batchify(corpus.test, eval_batch_size)

In [6]:
ntokens = len(corpus.dictionary)
model = MyRNNModel('RNN_TANH', ntokens, size_embedding, n_hidden_layers, n_rnn_layers, p_dropout).to(device)
criterion = nn.NLLLoss()

In [7]:
def repackage_hidden(h): 
    """Wraps hidden states in new Tensors, to detach them from their history."""

    if isinstance(h, torch.Tensor):
        return h.detach()
    else:
        return tuple(repackage_hidden(v) for v in h)

In [8]:
def get_batch(source, i):
    seq_len = min(sequence_length, len(source) - 1 - i)
    data = source[i:i+seq_len]
    target = source[i+1:i+1+seq_len].view(-1)
    return data, target


def evaluate(data_source):
    # Turn on evaluation mode which disables dropout.
    model.eval()
    total_loss = 0.
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(eval_batch_size)
    with torch.no_grad():
        for i in range(0, data_source.size(0) - 1, sequence_length):
            data, targets = get_batch(data_source, i)
            output, hidden = model(data, hidden)
            hidden = repackage_hidden(hidden)
            total_loss += len(data) * criterion(output, targets).item()
    return total_loss / (len(data_source) - 1)

In [9]:
def train():
    # Turn on training mode which enables dropout.
    model.train()
    total_loss = 0.
    start_time = time.time()
    ntokens = len(corpus.dictionary)
    hidden = model.init_hidden(train_batch_size)
    
    for batch, i in enumerate(range(0, train_data.size(0) - 1, sequence_length)):
        data, targets = get_batch(train_data, i)
        # Starting each batch, we detach the hidden state from how it was previously produced.
        # If we didn't, the model would try backpropagating all the way to start of the dataset.
        model.zero_grad()
        hidden = repackage_hidden(hidden)
        output, hidden = model(data, hidden)
        
        loss = criterion(output, targets)
        loss.backward()
        
        # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
#        torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        for p in model.parameters():
            p.data.add_(p.grad, alpha=-lr)

        total_loss += loss.item()

        if batch % log_interval == 0 and batch > 0:
            cur_loss = total_loss / log_interval
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches | lr {:02.2f} | ms/batch {:5.2f} | '
                    'loss {:5.2f} | ppl {:8.2f}'.format(
                epoch, batch, len(train_data) // sequence_length, lr,
                elapsed * 1000 / log_interval, cur_loss, math.exp(cur_loss)))
            total_loss = 0
            start_time = time.time()
#        if args.dry_run:
#            break


In [10]:
# Loop over epochs.
lr = 2.5
n_epochs = 20
best_val_loss = None
log_interval = 1

# At any point you can hit Ctrl + C to break out of training early.
try:
    for epoch in range(1, n_epochs+1):
        epoch_start_time = time.time()
        train()
        val_loss = evaluate(val_data)
        print('-' * 89)
        print('| end of epoch {:3d} | time: {:5.2f}s | valid loss {:5.2f} | '
                'valid ppl {:8.2f}'.format(epoch, (time.time() - epoch_start_time),
                                           val_loss, math.exp(val_loss)))
        print('-' * 89)
        # Save the model if the validation loss is the best we've seen so far.
        if not best_val_loss or val_loss < best_val_loss:
            with open('ex1_Elman.torch', 'wb') as f:
                torch.save(model, f)
            best_val_loss = val_loss
        else:
            # Anneal the learning rate if no improvement has been seen in the validation dataset.
            lr /= 4.0


except KeyboardInterrupt:
    print('-' * 89)
    print('Exiting from training early')

| epoch   1 |     1/  743 batches | lr 2.50 | ms/batch 332.69 | loss 18.32 | ppl 90292192.60
| epoch   1 |     2/  743 batches | lr 2.50 | ms/batch 20.19 | loss  8.94 | ppl  7642.21
| epoch   1 |     3/  743 batches | lr 2.50 | ms/batch 20.06 | loss  8.51 | ppl  4983.76
| epoch   1 |     4/  743 batches | lr 2.50 | ms/batch 19.91 | loss  8.00 | ppl  2972.74
| epoch   1 |     5/  743 batches | lr 2.50 | ms/batch 19.99 | loss  8.20 | ppl  3641.72
| epoch   1 |     6/  743 batches | lr 2.50 | ms/batch 20.04 | loss  8.09 | ppl  3275.08
| epoch   1 |     7/  743 batches | lr 2.50 | ms/batch 20.59 | loss  7.76 | ppl  2349.30
| epoch   1 |     8/  743 batches | lr 2.50 | ms/batch 21.72 | loss  7.58 | ppl  1949.78
| epoch   1 |     9/  743 batches | lr 2.50 | ms/batch 21.13 | loss  7.58 | ppl  1965.41
| epoch   1 |    10/  743 batches | lr 2.50 | ms/batch 21.23 | loss  7.53 | ppl  1868.93
| epoch   1 |    11/  743 batches | lr 2.50 | ms/batch 23.78 | loss  7.69 | ppl  2176.01
| epoch   1 |    

| epoch   1 |    93/  743 batches | lr 2.50 | ms/batch 20.29 | loss  8.25 | ppl  3811.81
| epoch   1 |    94/  743 batches | lr 2.50 | ms/batch 20.79 | loss 10.64 | ppl 41593.46
| epoch   1 |    95/  743 batches | lr 2.50 | ms/batch 20.05 | loss  8.00 | ppl  2991.05
| epoch   1 |    96/  743 batches | lr 2.50 | ms/batch 19.92 | loss  8.30 | ppl  4017.05
| epoch   1 |    97/  743 batches | lr 2.50 | ms/batch 19.81 | loss  8.07 | ppl  3210.38
| epoch   1 |    98/  743 batches | lr 2.50 | ms/batch 19.69 | loss  8.32 | ppl  4114.10
| epoch   1 |    99/  743 batches | lr 2.50 | ms/batch 19.96 | loss  9.55 | ppl 14027.20
| epoch   1 |   100/  743 batches | lr 2.50 | ms/batch 19.88 | loss 11.69 | ppl 119214.73
| epoch   1 |   101/  743 batches | lr 2.50 | ms/batch 20.02 | loss  8.80 | ppl  6644.55
| epoch   1 |   102/  743 batches | lr 2.50 | ms/batch 20.10 | loss  9.15 | ppl  9458.58
| epoch   1 |   103/  743 batches | lr 2.50 | ms/batch 19.96 | loss  8.72 | ppl  6099.79
| epoch   1 |   104/

| epoch   1 |   190/  743 batches | lr 2.50 | ms/batch 21.37 | loss  7.39 | ppl  1625.37
| epoch   1 |   191/  743 batches | lr 2.50 | ms/batch 20.41 | loss  7.68 | ppl  2156.51
| epoch   1 |   192/  743 batches | lr 2.50 | ms/batch 20.03 | loss  7.92 | ppl  2758.47
| epoch   1 |   193/  743 batches | lr 2.50 | ms/batch 19.80 | loss  7.49 | ppl  1788.66
| epoch   1 |   194/  743 batches | lr 2.50 | ms/batch 19.75 | loss  7.61 | ppl  2024.13
| epoch   1 |   195/  743 batches | lr 2.50 | ms/batch 19.90 | loss  7.89 | ppl  2660.06
| epoch   1 |   196/  743 batches | lr 2.50 | ms/batch 20.11 | loss  7.63 | ppl  2053.77
| epoch   1 |   197/  743 batches | lr 2.50 | ms/batch 20.01 | loss  7.44 | ppl  1695.13
| epoch   1 |   198/  743 batches | lr 2.50 | ms/batch 19.87 | loss  7.16 | ppl  1280.80
| epoch   1 |   199/  743 batches | lr 2.50 | ms/batch 19.97 | loss  7.33 | ppl  1525.37
| epoch   1 |   200/  743 batches | lr 2.50 | ms/batch 19.72 | loss  7.08 | ppl  1188.60
| epoch   1 |   201/ 

| epoch   1 |   292/  743 batches | lr 2.50 | ms/batch 26.43 | loss  6.75 | ppl   850.63
| epoch   1 |   293/  743 batches | lr 2.50 | ms/batch 21.20 | loss  6.88 | ppl   975.24
| epoch   1 |   294/  743 batches | lr 2.50 | ms/batch 22.92 | loss  6.89 | ppl   986.70
| epoch   1 |   295/  743 batches | lr 2.50 | ms/batch 22.22 | loss  6.81 | ppl   909.14
| epoch   1 |   296/  743 batches | lr 2.50 | ms/batch 21.50 | loss  6.90 | ppl   988.33
| epoch   1 |   297/  743 batches | lr 2.50 | ms/batch 22.40 | loss  6.89 | ppl   977.50
| epoch   1 |   298/  743 batches | lr 2.50 | ms/batch 20.71 | loss  7.15 | ppl  1272.01
| epoch   1 |   299/  743 batches | lr 2.50 | ms/batch 20.44 | loss  7.02 | ppl  1121.56
| epoch   1 |   300/  743 batches | lr 2.50 | ms/batch 20.69 | loss  6.95 | ppl  1045.59
| epoch   1 |   301/  743 batches | lr 2.50 | ms/batch 20.98 | loss  6.82 | ppl   918.31
| epoch   1 |   302/  743 batches | lr 2.50 | ms/batch 21.60 | loss  6.81 | ppl   906.83
| epoch   1 |   303/ 

| epoch   1 |   394/  743 batches | lr 2.50 | ms/batch 23.17 | loss  6.72 | ppl   828.83
| epoch   1 |   395/  743 batches | lr 2.50 | ms/batch 22.00 | loss  6.80 | ppl   901.74
| epoch   1 |   396/  743 batches | lr 2.50 | ms/batch 21.79 | loss  7.00 | ppl  1094.78
| epoch   1 |   397/  743 batches | lr 2.50 | ms/batch 21.69 | loss  6.84 | ppl   934.80
| epoch   1 |   398/  743 batches | lr 2.50 | ms/batch 20.85 | loss  6.97 | ppl  1062.06
| epoch   1 |   399/  743 batches | lr 2.50 | ms/batch 20.75 | loss  6.85 | ppl   945.40
| epoch   1 |   400/  743 batches | lr 2.50 | ms/batch 20.67 | loss  6.85 | ppl   948.32
| epoch   1 |   401/  743 batches | lr 2.50 | ms/batch 20.43 | loss  6.68 | ppl   798.44
| epoch   1 |   402/  743 batches | lr 2.50 | ms/batch 20.61 | loss  6.78 | ppl   878.09
| epoch   1 |   403/  743 batches | lr 2.50 | ms/batch 20.52 | loss  6.82 | ppl   913.16
| epoch   1 |   404/  743 batches | lr 2.50 | ms/batch 21.29 | loss  6.86 | ppl   954.71
| epoch   1 |   405/ 

...

| epoch  20 |   295/  743 batches | lr 0.62 | ms/batch 24.70 | loss  6.07 | ppl   434.51
| epoch  20 |   296/  743 batches | lr 0.62 | ms/batch 23.58 | loss  6.06 | ppl   430.22
| epoch  20 |   297/  743 batches | lr 0.62 | ms/batch 21.46 | loss  5.91 | ppl   369.60
| epoch  20 |   298/  743 batches | lr 0.62 | ms/batch 21.40 | loss  6.17 | ppl   478.04
| epoch  20 |   299/  743 batches | lr 0.62 | ms/batch 21.71 | loss  5.99 | ppl   398.81
| epoch  20 |   300/  743 batches | lr 0.62 | ms/batch 21.83 | loss  6.07 | ppl   432.18
| epoch  20 |   301/  743 batches | lr 0.62 | ms/batch 21.87 | loss  5.99 | ppl   397.91
| epoch  20 |   302/  743 batches | lr 0.62 | ms/batch 21.47 | loss  5.97 | ppl   390.76
| epoch  20 |   303/  743 batches | lr 0.62 | ms/batch 21.61 | loss  5.98 | ppl   396.99
| epoch  20 |   304/  743 batches | lr 0.62 | ms/batch 21.58 | loss  6.00 | ppl   404.28
| epoch  20 |   305/  743 batches | lr 0.62 | ms/batch 21.67 | loss  5.99 | ppl   400.01
| epoch  20 |   306/ 

| epoch  20 |   393/  743 batches | lr 0.62 | ms/batch 26.12 | loss  6.08 | ppl   437.39
| epoch  20 |   394/  743 batches | lr 0.62 | ms/batch 27.31 | loss  5.95 | ppl   384.36
| epoch  20 |   395/  743 batches | lr 0.62 | ms/batch 24.55 | loss  6.02 | ppl   409.54
| epoch  20 |   396/  743 batches | lr 0.62 | ms/batch 25.00 | loss  6.19 | ppl   490.07
| epoch  20 |   397/  743 batches | lr 0.62 | ms/batch 24.98 | loss  5.86 | ppl   352.42
| epoch  20 |   398/  743 batches | lr 0.62 | ms/batch 24.61 | loss  6.01 | ppl   407.89
| epoch  20 |   399/  743 batches | lr 0.62 | ms/batch 25.15 | loss  6.00 | ppl   403.72
| epoch  20 |   400/  743 batches | lr 0.62 | ms/batch 25.50 | loss  6.04 | ppl   421.17
| epoch  20 |   401/  743 batches | lr 0.62 | ms/batch 24.49 | loss  5.95 | ppl   383.06
| epoch  20 |   402/  743 batches | lr 0.62 | ms/batch 30.91 | loss  6.04 | ppl   419.08
| epoch  20 |   403/  743 batches | lr 0.62 | ms/batch 27.87 | loss  5.99 | ppl   398.43
| epoch  20 |   404/ 

| epoch  20 |   491/  743 batches | lr 0.62 | ms/batch 24.32 | loss  6.04 | ppl   421.82
| epoch  20 |   492/  743 batches | lr 0.62 | ms/batch 28.16 | loss  5.99 | ppl   400.53
| epoch  20 |   493/  743 batches | lr 0.62 | ms/batch 26.61 | loss  6.08 | ppl   437.13
| epoch  20 |   494/  743 batches | lr 0.62 | ms/batch 22.19 | loss  6.06 | ppl   428.06
| epoch  20 |   495/  743 batches | lr 0.62 | ms/batch 22.22 | loss  5.93 | ppl   375.24
| epoch  20 |   496/  743 batches | lr 0.62 | ms/batch 23.25 | loss  6.06 | ppl   429.37
| epoch  20 |   497/  743 batches | lr 0.62 | ms/batch 22.04 | loss  5.93 | ppl   376.42
| epoch  20 |   498/  743 batches | lr 0.62 | ms/batch 22.00 | loss  6.14 | ppl   463.70
| epoch  20 |   499/  743 batches | lr 0.62 | ms/batch 24.23 | loss  6.00 | ppl   403.20
| epoch  20 |   500/  743 batches | lr 0.62 | ms/batch 24.46 | loss  5.90 | ppl   364.76
| epoch  20 |   501/  743 batches | lr 0.62 | ms/batch 26.38 | loss  6.01 | ppl   409.28
| epoch  20 |   502/ 

| epoch  20 |   592/  743 batches | lr 0.62 | ms/batch 23.98 | loss  5.95 | ppl   382.83
| epoch  20 |   593/  743 batches | lr 0.62 | ms/batch 23.44 | loss  5.94 | ppl   378.88
| epoch  20 |   594/  743 batches | lr 0.62 | ms/batch 22.89 | loss  5.90 | ppl   364.03
| epoch  20 |   595/  743 batches | lr 0.62 | ms/batch 22.74 | loss  5.92 | ppl   371.65
| epoch  20 |   596/  743 batches | lr 0.62 | ms/batch 22.42 | loss  5.85 | ppl   348.83
| epoch  20 |   597/  743 batches | lr 0.62 | ms/batch 22.27 | loss  5.85 | ppl   346.62
| epoch  20 |   598/  743 batches | lr 0.62 | ms/batch 23.29 | loss  5.96 | ppl   386.42
| epoch  20 |   599/  743 batches | lr 0.62 | ms/batch 22.16 | loss  6.08 | ppl   435.03
| epoch  20 |   600/  743 batches | lr 0.62 | ms/batch 22.19 | loss  5.89 | ppl   359.89
| epoch  20 |   601/  743 batches | lr 0.62 | ms/batch 22.45 | loss  6.03 | ppl   414.20
| epoch  20 |   602/  743 batches | lr 0.62 | ms/batch 26.08 | loss  5.96 | ppl   388.01
| epoch  20 |   603/ 

| epoch  20 |   685/  743 batches | lr 0.62 | ms/batch 27.81 | loss  5.98 | ppl   395.03
| epoch  20 |   686/  743 batches | lr 0.62 | ms/batch 36.77 | loss  5.99 | ppl   399.36
| epoch  20 |   687/  743 batches | lr 0.62 | ms/batch 32.81 | loss  5.94 | ppl   379.48
| epoch  20 |   688/  743 batches | lr 0.62 | ms/batch 32.47 | loss  6.05 | ppl   423.90
| epoch  20 |   689/  743 batches | lr 0.62 | ms/batch 27.19 | loss  6.05 | ppl   424.42
| epoch  20 |   690/  743 batches | lr 0.62 | ms/batch 28.54 | loss  6.02 | ppl   409.80
| epoch  20 |   691/  743 batches | lr 0.62 | ms/batch 26.62 | loss  5.89 | ppl   359.67
| epoch  20 |   692/  743 batches | lr 0.62 | ms/batch 27.09 | loss  6.10 | ppl   447.91
| epoch  20 |   693/  743 batches | lr 0.62 | ms/batch 31.83 | loss  6.02 | ppl   410.26
| epoch  20 |   694/  743 batches | lr 0.62 | ms/batch 31.16 | loss  5.98 | ppl   395.66
| epoch  20 |   695/  743 batches | lr 0.62 | ms/batch 30.53 | loss  6.01 | ppl   406.40
| epoch  20 |   696/ 

In [11]:
corpus

In [12]:
corpus.train.shape

torch.Size([929589])

In [13]:
corpus.test.shape

torch.Size([82430])

In [14]:
corpus.valid.shape

torch.Size([73760])

In [15]:
len(corpus.dictionary.idx2word)

10000

In [16]:
corpus.dictionary.word2idx['<unk>']

26

In [17]:
corpus.dictionary.word2idx['<eos>']

24

In [18]:
corpus.

SyntaxError: invalid syntax (1008075712.py, line 1)

There is a lot more that you can do with outputs (such as including interactive outputs)
with your book. For more information about this, see [the Jupyter Book documentation](https://jupyterbook.org)

In [21]:
from mymodel import MyElmanNetwork
import torch.optim as optim

# Example usage
vocab_size = 10000
embedding_dim = 100
input_size = embedding_dim
hidden_size = 20
output_size = 5
sequence_length = 6
batch_size = 3
num_epochs = 10
learning_rate = 0.001

# Create an instance of the Elman network
elman_net = MyElmanNetwork(input_size, hidden_size, output_size, vocab_size, embedding_dim)

# Generate some random input sequence and corresponding target labels
input_sequence = torch.randint(0, vocab_size, (batch_size, sequence_length))
target_labels = torch.randint(0, output_size, (batch_size, sequence_length))

# Define the loss function and the optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(elman_net.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    # Zero the gradients
    optimizer.zero_grad()

    # Forward pass
    outputs = elman_net(input_sequence)

    # Reshape outputs and target labels for loss calculation
    outputs = outputs.view(-1, output_size)
    target_labels = target_labels.view(-1)

    # Calculate the loss
    loss = loss_function(outputs, target_labels)

    # Backward pass and optimization step
    loss.backward()
    optimizer.step()

    # Print the loss for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

# Print the final outputs
final_outputs = elman_net(input_sequence)
print(final_outputs)

Epoch 1/10, Loss: 1.6084262132644653
Epoch 2/10, Loss: 1.5708050727844238
Epoch 3/10, Loss: 1.5337529182434082
Epoch 4/10, Loss: 1.4973175525665283
Epoch 5/10, Loss: 1.4615721702575684
Epoch 6/10, Loss: 1.4265872240066528
Epoch 7/10, Loss: 1.3924317359924316
Epoch 8/10, Loss: 1.3591649532318115
Epoch 9/10, Loss: 1.326827883720398
Epoch 10/10, Loss: 1.2954357862472534
tensor([[[-0.4557,  0.3464,  0.6389,  0.3665, -0.7939],
         [ 0.1077,  0.0440,  0.5277,  0.1991,  0.1102],
         [-0.2033,  0.5700, -0.0068,  0.3846,  0.2028],
         [ 0.2616, -0.4129,  0.1387,  0.0085,  0.7012],
         [-0.4419,  0.3845,  0.0186,  0.4997, -0.4066],
         [-0.2927,  0.0781,  0.9332,  0.6109, -0.3657]],

        [[-0.0044, -0.1858,  0.0045,  0.6286,  0.2181],
         [ 0.0687,  0.1818, -0.2695,  0.2359, -0.0188],
         [-0.5020, -0.2445,  0.2477,  0.6203, -0.5808],
         [-0.6771, -0.3951,  0.8126,  0.3976, -0.4112],
         [-0.0098,  0.5429, -0.4774,  0.2871, -0.4462],
         [ 0